## Issue with the different probe design in BAV project

There are two different probe design i.e Probe_deisgn4_5 and Probe_design_6. Using these probes design gives completely different interaction calls for all the replicates, since earlier two biological replicates were used to generate the interaction while the third biological replicate HiCap interaction was done using probe design 6. Thus to deal with this issue, we had a meeting regarding this. The main resolutions of the meeting are as follows:

* Use Deisgn 6 as the univeral probe design
* Improve the negative control probe design using both design4_5 and design6

Pseudo-code for script for fixing the issues:

* Use the Design6 Transcipt file and SNV probe deisng files 
* For the transcript file, cluster the promoter isoforms (d = 1200 bp)
    * for (+) strand ---> take the left most TSS
    * for (-) strand ---> take the right most TSS
* Generate the mock probe file named as MergedMockDesign4_5_6
* Merge Negative control for Probe_design4_5 and probedesign6


## Part 1 Clustering of the promoters at 1200 bp

In [1]:
## Part1 : Reading the probedeisngn6 "Sorted_Ref_NC_ENSEMBL-transcropt.bed6sorted.dedup.txt" 
## and generating the new transcript file wherein the promoter within 1200bp are clustered together
## and generating a merged probe 4,5 and 6 transcript file


'''
To do this first we gonna try few things:

    * We first divide genes based on the strand
    * We then make a dictionary based start position of the gene 
    * The key here will be the position of the gene and values will be the other fields

'''

'\nTo do this first we gonna try few things:\n\n    * We first divide genes based on the strand\n    * We then make a dictionary based start position of the gene \n    * The key here will be the position of the gene and values will be the other fields\n\n'

### "-" strand manipulation

In [2]:
import os
os.chdir ("/Volumes/Work_drive/prj/BAV_TAV/data/raw_internal/Feature_probe_file")
probe_file = "Sorted_Ref_PC_NC_ENSEMBL_transcript.bed6.sorted.dedup.txt"

## Generating a dictionary of probe_transcript that have the "-" strand annotation 

pro_tran_dict = {}
with open (probe_file, "r") as probe:
    for lines in probe:
        line = lines.strip().split("\t")
        if line[5]  == "-":
            postion_combine = line[0]+":"+line[1]+":"+line[2]+":"+line[3]+":"+"."+":"+line[5]
            combo = pro_tran_dict.get(postion_combine, [])
            test = line[6]
            combo.append(test)
            pro_tran_dict[postion_combine] = combo            

len_con = []
for keys, values in pro_tran_dict.items():
    new_keys = keys+":"+values[0]
    len_con.append(new_keys)
    

test = {}
for ele in len_con:
    break_set = ele.split(":")
    gene = break_set [3]
    distance_all = test.get(gene, [])
    few_all = break_set[0],break_set[1],break_set[2],break_set[4],break_set[5],break_set[6]
    distance_all.append(few_all)
    test[gene] = distance_all 

print (len (test.keys()))


10866


In [3]:
import numpy as np

all_trancript = []
file_out = "test_2.txt"
fh = open (file_out, "w")

for keys, values in test.items():
    if len(values) >= 2:
        nes = {}
        distance = []
        
        for element in values:
            position = element[2]
            combo = nes.get(position, '')
            combo = element[0], element[1],element[2],element[3],element[4],element[5]
            nes[position] = combo
            distance.append(position)
            
            counter = 0 
            loop_counter = [0]
            #print(distance)
            new_dist = distance.copy()
            mapping_dict ={}
            
            
            for ele in distance:
                #print(ele)
                new_dist[counter] = int(ele)+1200
                dist = int(ele)+1200
                
                newest = [v for i,v in enumerate(distance) if int(v) < dist ]
                max_val = max(newest)
                
                counter=counter+1
                new_dist = distance.copy()
                #print(new_dist)
                #print(max_val)
                almost_there = mapping_dict.get(ele, '')
                almost_there = str(max_val)
                mapping_dict[ele] = almost_there 
                
        uniq_dist = np.unique(list(mapping_dict.values()))
                
        #print(uniq_dist)
            
        for dis in uniq_dist:
            fh.write ("\t".join(nes[dis])+"\t"+keys)
            fh.write("\n")
            
            
            
    elif len(values) < 2:
        for element in values:
            all = []
            
            all.append (element[0])
            all.append (element[1])
            all.append (element[2])
            all.append (element[3])
            all.append (element[4])
            all.append (element[5])
            all.append(keys)
        
            fh.write("\t".join(all))
            fh.write("\n")
fh.close()

### "+" strand manipulation

In [4]:
import os
os.chdir ("/Volumes/Work_drive/prj/BAV_TAV/data/raw_internal/Feature_probe_file")

probe_file = "Sorted_Ref_PC_NC_ENSEMBL_transcript.bed6.sorted.dedup.txt"

pro_tran_dict = {}
with open (probe_file, "r") as probe:
    for lines in probe:
        line = lines.strip().split("\t")
        if line[5]  == "+":
            postion_combine = line[0]+":"+line[1]+":"+line[2]+":"+line[3]+":"+"."+":"+line[5]
            combo = pro_tran_dict.get(postion_combine, [])
            test = line[6]
            combo.append(test)
            pro_tran_dict[postion_combine] = combo            

len_con = []

for keys, values in pro_tran_dict.items():
    new_keys = keys+":"+values[0]
    len_con.append(new_keys)
    

test = {}

for ele in len_con:
    break_set = ele.split(":")
    gene = break_set [3]
    distance_all = test.get(gene, [])
    few_all = break_set[0],break_set[1],break_set[2],break_set[4],break_set[5],break_set[6]
    distance_all.append(few_all)
    test[gene] = distance_all 

print (len(list(test.keys())))

11166


In [5]:
import numpy as np

file_out = "test_2_+_strand.txt"
fh = open (file_out, "w")

for keys, values in test.items():
    if len(values) >= 2:
        nes = {}
        distance = []
        
        for element in values:
            position = element[1]
            combo = nes.get(position, '')
            combo = element[0], element[1],element[2],element[3],element[4],element[5]
            nes[position] = combo
            distance.append(position)
            
            counter = 0 
            loop_counter = [0]
            #print(distance)
            new_dist = distance.copy()
            mapping_dict ={}
            
            
            for ele in distance:
                #print(ele)
                new_dist[counter] = int(ele)-1200
                dist = int(ele)-1200
                
                newest = [v for i,v in enumerate(distance) if int(v) > dist ]
                max_val = min(newest)
                
                counter=counter+1
                new_dist = distance.copy()
                #print(new_dist)
                #print(max_val)
                almost_there = mapping_dict.get(ele, '')
                almost_there = str(max_val)
                mapping_dict[ele] = almost_there 
                
        uniq_dist = np.unique(list(mapping_dict.values()))
                
        #print(uniq_dist)
            
        for dis in uniq_dist:
            fh.write ("\t".join(nes[dis])+"\t"+keys)
            fh.write("\n")
    
    elif len(values) < 2:
        for element in values:
            all = []
            
            all.append (element[0])
            all.append (element[1])
            all.append (element[2])
            all.append (element[3])
            all.append (element[4])
            all.append (element[5])
            all.append(keys)
            #all.append(keys)
            fh.write("\t".join(all))
            fh.write("\n")
fh.close()
            

## Part2 

In [6]:
%%bash

awk -F "\t" -v OFS="\t" '{print $1,$2,$3,$7,$4,$5,$6,$7}' test_2.txt > tmp; mv tmp test_2.txt 
awk -F "\t" -v OFS="\t" '{print $1,$2,$3,$7,$4,$5,$6,$7}' test_2_+_strand.txt > tmp; mv tmp test_2_+_strand.txt
cat test_2.txt test_2_+_strand.txt | sort  -k4,4 > Design6_hg19_probe_combined.txt
#rm -r test_2.txt test_2_+_strand.txt

In [1]:
import os
os.chdir ("/Volumes/Work_drive/prj/BAV_TAV/data/raw_internal/Feature_probe_file")

file_out = "test.gff3"
fh = open (file_out, "w")

header1= "##gff-version 3.2.1"
header2 = "##genome-build UCSC hg19"

fh.write(header1)
fh.write("\n")
fh.write(header2)
fh.write("\n")

target_set = ("promoter", "SNP")

combined_deisgn = "Design6_hg19_probe_combined.txt"
with open (combined_deisgn, "r") as design:
    for lines in design:
        line = lines.strip().split("\t")
        
        probe = "probe"
        transcript = "transcriptid="+line[6]+";"
        side = ("R","L")
        target = "target="+target_set[0]+";"
        design_name = "design=MergedMockDesign;"
        feature_vicinity = "featuresinvicinity=none;"
        name = "Name="+line[-1]+";"
        distance_tss ="distancetotss=2500"
        
        #print (line)
        #break
        if line[5]  == "+":
            #print (line)
            Right_probe_end = int(line[1])+2500
            #Right_probe_start = Right_probe_end-120
            Right_probe_start = int(line[1])
            Left_probe_start = int(line[1])-2500
            #Left_probe_end = Left_probe_start +120
            Left_probe_end = int(line[1])
            tss_target = "targettss="+line[1]+";"
            
            #print(side)
            
            for side in side:
                if side == "R":
                    side1 = "side=R;"
                    all_name = [name, transcript, 
                                side1,target,design_name,
                                feature_vicinity,tss_target,distance_tss]
                    fewest = line[0]+"\t"+"."+"\t"+probe+"\t"+str(Right_probe_start)+"\t"+str(Right_probe_end)+"\t"+"."+"\t"+"."+"\t"+"."+"\t"+" ".join(all_name)
                    fh.write (fewest)
                    fh.write("\n")
                
                else:
                    side1 = "side=L;"
                    all_name = [name, transcript, 
                                side1,target,design_name,
                                feature_vicinity,tss_target,distance_tss]
                    fewest = line[0]+"\t"+"."+"\t"+probe+"\t"+str(Left_probe_start)+"\t"+str(Left_probe_end)+"\t"+"."+"\t"+"."+"\t"+"."+"\t"+" ".join(all_name)
                    fh.write (fewest)
                    fh.write("\n")
                    
        else:
            #print (line)
            Right_probe_end = int(line[2])+2500
            #Right_probe_start = Right_probe_end-120
            Right_probe_start = int(line[2])
            Left_probe_start = int(line[2])-2500
            #Left_probe_end = Left_probe_start +120
            Left_probe_end = int(line[2])
            tss_target = "targettss="+line[2]+";"
            
            #print(side)
            
            for side in side:
                if side == "R":
                    side1 = "side=R;"
                    all_name = [name, transcript, 
                                side1,target,design_name,
                                feature_vicinity,tss_target,distance_tss]
                    fewest = line[0]+"\t"+"."+"\t"+probe+"\t"+str(Right_probe_start)+"\t"+str(Right_probe_end)+"\t"+"."+"\t"+"."+"\t"+"."+"\t"+" ".join(all_name)
                    fh.write (fewest)
                    fh.write("\n")
                
                else:
                    side1 = "side=L;"
                    all_name = [name, transcript, 
                                side1,target,design_name,
                                feature_vicinity,tss_target,distance_tss]
                    fewest = line[0]+"\t"+"."+"\t"+probe+"\t"+str(Left_probe_start)+"\t"+str(Left_probe_end)+"\t"+"."+"\t"+"."+"\t"+"."+"\t"+" ".join(all_name)
                    fh.write (fewest)
                    fh.write("\n")
                                
fh.close()

In [ ]:
fh = open (file_out, "a")
file_out = "test.gff3"

combined_deisgn = "Design6_GWASSNPs6.txt"
target_set = ("promoter", "SNP")

chr_all = ["chr1","chr2","chr3","chr4","chr5",
           "chr6","chr7","chr8","chr9","chr10",
           "chr11","chr12","chr13","chr14","chr15",
           "chr16","chr17","chr18","chr19","chr20",
           "chr21","chr22","chrX","chrY"]

target_set = ("promoter", "SNP")

with open (combined_deisgn, "r") as design:
    for lines in design:
        line = lines.strip().split("\t")
        
        #print (line)     
        probe = "probe"
        transcript = "transcriptid="+line[-1]+";"
        side = ("R","L")
        target = "target="+target_set[1]+";"
        design_name = "design=MergedMockDesign;"
        feature_vicinity = "featuresinvicinity=none;"
        name = "Name="+line[3]+";"
        distance_tss ="distancetotss=2500"
        
        Right_probe_end = int(line[1])+2500
        #Right_probe_start = Right_probe_end-120
        Right_probe_start = int(line[1])
        Left_probe_start = int(line[1])-2500
        #Left_probe_end = Left_probe_start +120
        Left_probe_end = int(line[1])
        tss_target = "targettss="+line[1]+";"
        
        for side in side:
            if side == "R" and line[0] in chr_all:
                side1 = "side=R;"
                all_name = [name, transcript, 
                            side1,target,design_name,
                            feature_vicinity,tss_target,distance_tss]
                fewest = line[0]+"\t"+"."+"\t"+probe+"\t"+str(Right_probe_start)+"\t"+str(Right_probe_end)+"\t"+"."+"\t"+"."+"\t"+"."+"\t"+" ".join(all_name)
                fh.write (fewest)
                fh.write("\n")
                
            elif side == "L" and line[0] in chr_all:
                side1 = "side=L" 
                all_name = [name, transcript, 
                            side1,target,design_name,
                            feature_vicinity,tss_target,distance_tss]
                fewest = line[0]+"\t"+"."+"\t"+probe+"\t"+str(Left_probe_start)+"\t"+str(Left_probe_end)+"\t"+"."+"\t"+"."+"\t"+"."+"\t"+" ".join(all_name)
                fh.write (fewest)
                fh.write("\n")
        
fh.close()

## gff3 manipulation for the negative control

In [24]:
%%bash

sed 's/Design4_Design5/MergedMockDesign/g' Probes_Design4_5.uniq.exonspurged.NegCtrl.gff3 > tmp1.gff3
sed '1,2d'  Design6.hg19.NegCtrlProbes.MboI.15.37.32_2017-08-25.1250.gff3  | sed 's/Design6/MergedMockDesign/g'  > tmp2.gff3

cat  tmp1.gff3 tmp2.gff3 > Merged4.5.6.hg19.NegCtrlProbes.MboI.2019-01-16.gff3

mv test.gff3 Merged4.5.6.hg19.AllProbes.MboI.2019-01-16.gff3
rm -r tmp1.gff3 tmp2.gff3

mv: test.gff3: No such file or directory


In [21]:
2129053+104216145

106345198

In [25]:
2243933 - 2220111

23822

/crex/proj/g2018023/nobackup/private/

### TAV specific interaction


MLLT10  NM_001324297  MLLT10_21822277 chr10 21822277  22032559  1 + chr10 22618836  22619031  796559  8 2.18662e-06 0_0_4_4 16  0 0_0_4_12  9 0.000442346 0_0_9_0 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0
LMO3  NM_001243612  LMO3_16760084 chr12 16760084  16701305  1 - chr12 16079620  16080013  -680464 7 0.000105111 4_0_3_0 9 9.33002e-08 0_0_3_6 11  1.85752e-05 0_3_0_8 0 1 0_0_0_0 0 1 0_0_0_0 0
LMO3  NM_001243612  LMO3_16760084 chr12 16760084  16701305  1 - chr12 18217539  18218100  1457455 6 0.000139611 0_0_3_3 6 0.000156767 0_0_3_3 9 0.000211407 0_0_9_0 0 1 0_0_0_0 0 1 0_0_0_0 0 1
FLRT2 NM_001346144  FLRT2_85996368  chr14 85996368  86095040  1 + chr14 85226825  85227331  -769543 6 0.000704606 0_0_6_0 6 0.000531067 0_0_3_3 9 0.000449257 0_0_0_9 0 1 0_0_0_0 0 1 0_0_0_0 0
rs2121070 _Blood_pressure_gwa rs2121070_76650763  chr14 76650763  76650764  2 + chr14 76525698  76526537  -125065 18  6.668e-13 2_0_8_8 8 0.0008502 0_2_4_2 20  1.00568e-10 0_0_10_10 0 1 0_0_0
LRRC46  NM_033413 LRRC46_45908992 chr17 45908992  45915079  1 + chr17 75154207  75186682  29245215  6 0.000113597 0_6_0_0 8 8.21765e-08 2_6_0_0 9 9.06283e-05 0_9_0_0 0 1 0_0_0_0 0 1 0_0_0_0 0
MRPL10  NM_145255 MRPL10_45908907 chr17 45908907  45900637  1 - chr17 75154207  75186682  29245300  6 0.000113597 0_6_0_0 8 8.21765e-08 2_6_0_0 9 9.06283e-05 0_9_0_0 0 1 0_0_0_0 0 1 0_0_0_0 0
ZNF532  NM_001318726  ZNF532_56529831 chr18 56529831  56653712  1 + chr18 56745683  56745796  215852  12  1.55933e-07 5_0_7_0 19  0 5_10_4_0  12  4.23337e-05 5_0_7_0 0 1 0_0_0_0 0 1 0_0_0_0 0
FSTL1 NM_007085 FSTL1_120169918 chr3  120169918 120113060 1 - chr3  119365819 119366969 -804099 8 2.05506e-06 0_0_4_4 7 3.22176e-05 2_1_4_0 11  1.11704e-05 1_0_6_4 0 1 0_0_0_0 0 1 0_0_0_0 0 1
SULT1E1 NM_005420 SULT1E1_70725870  chr4  70725870  70706929  1 - chr4  73836136  73842479  3110266 6 7.84936e-05 0_2_0_4 9 3.75791e-10 0_1_4_4 8 0.000578896 0_0_0_8 0 1 0_0_0_0 0 1 0_0_0_0 0
GRXCR2  NM_001080516  GRXCR2_145252531  chr5  145252531 145239295 1 - chr5  145461080 145461770 208549  8 0.000899318 8_0_0_0 10  2.0521e-06  4_6_0_0 14  1.36521e-06 2_6_0_6 0 1 0_0_0_0 0 1 0
(END)


### BAV specific interaction

rs4590817 _Blood_pressure_gwa rs4590817_63467552  chr10 63467552  63467553  2 + chr10 62584203  62584829  -883349 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 6 0.000892326 0_0_4_2 6 0.000509739 0_0_0
ZEB1  NM_001323638  ZEB1_31607423 chr10 31607423  31818742  1 + chr10 31980261  31980481  372838  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 17  0 6_0_0_11  36  0 6_0_6_24  16  5.443e-10 0_0_16_0
ZEB1  NM_001323638  ZEB1_31607423 chr10 31607423  31818742  1 + chr10 31984441  31984832  377018  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 12  2.95248e-09 0_0_0_12  12  5.36649e-11 6_6_0_0 12  6.0
ZEB1  NM_001323638  ZEB1_31607423 chr10 31607423  31818742  1 + chr10 32269586  32270182  662163  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 12  6.74949e-12 6_0_0_6 6 0.000865634 0_0_0_6 9 0.0005489
BUD13 NM_001159736  BUD13_116643714 chr11 116643714 116618885 1 - chr11 113983708 113984331 -2660006  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 6 8.68203e-05 0_4_2_0 6 0.000101541 0_2_2_2 8 0.00073
PLEKHB1 NM_001130033  PLEKHB1_73357222  chr11 73357222  73373864  1 + chr11 73835512  73836155  478290  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 12  2.86965e-10 0_7_1_4 8 1.10751e-05 0_0_4_4 9 0.0
LINC00615 ENST00000546725.1 LINC00615_91311799  chr12 91311799  91342446  1 + chr12 90354468  90355368  -957331 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 8 5.24398e-06 2_0_6_0 8 1.30979e-06 2_0_6_0
LMO3  NM_001243612  LMO3_16760084 chr12 16760084  16701305  1 - chr12 15477898  15479170  -1282186  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 15  0 0_0_12_3  9 9.74993e-09 0_0_6_3 9 0.000225795 0_0
rs2072376 _GM12878_C_gwa  rs2072376_6579811 chr12 6579811 6579812 2 + chr12 6712107 6713219 132296  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 12  9.35246e-06 0_0_6_6 8 0.000702201 0_0_4_4 12  0.000
SLC38A1 NM_001278389  SLC38A1_46660278  chr12 46660278  46576839  1 - chr12 45938038  45940453  -722240 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 12  1.28497e-12 6_3_0_3 6 0.000671138 0_0_0_6 9 0.0
BMP4  NM_130851 BMP4_54421768 chr14 54421768  54416454  1 - chr14 56806154  56811379  2384386 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 10  6.35225e-12 5_0_5_0 6 0.000110887 6_0_0_0 8 0.000845193 0
DAD1  NM_001344 DAD1_23058143 chr14 23058143  23033806  1 - chr14 23344512  23345688  286369  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 8 0.00044439  3_0_4_1 7 0.000597236 0_0_3_4 12  1.51532e-05 0
MYO5A NM_000259 MYO5A_52821247  chr15 52821247  52599479  1 - chr15 48683781  48687120  -4137466  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 6 8.76093e-05 0_2_2_2 8 4.41544e-08 4_0_4_0 8 0.000625279
CAB39 NM_016289 CAB39_231577556 chr2  231577556 231685790 1 + chr2  234393366 234398631 2815810 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 11  1.88738e-14 0_1_4_6 10  8.80385e-12 0_0_6_4 8 0.0008868
LINC01114 ENST00000424321.5 LINC01114_105372213 chr2  105372213 105363095 1 - chr2  105424944 105425218 52731 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 12  0.000441499 0_0_3_9 14  7.14648e-08 0_2_3
ECE2  NM_001037324  ECE2_183993798  chr3  183993798 184010819 1 + chr3  183637207 183638337 -356591 0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 8 0.000209257 2_2_4_0 8 3.60621e-05 0_0_6_2 16  7.29658
SMC4  NM_001002800  SMC4_160117091  chr3  160117091 160152749 1 + chr3  158490041 158493436 -1627050  0 1 0_0_0_0 0 1 0_0_0_0 0 1 0_0_0_0 6 0.000180506 0_0_0_6 6 0.000175554 0_0_3_3 12  2.587
(END)


https://www.ahajournals.org/doi/10.1161/CIR.0000000000000606